In [3]:
pip install torch torchvision gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 27.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 88.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 设定数据转换：标准化并转换为Tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 标准化处理
])

# 下载并加载 CIFAR-10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:11<00:00, 14.6MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
import torch.nn as nn
import torch.optim as optim

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 定义卷积层，激活层，池化层等
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)  # 输入大小取决于图像尺寸
        self.fc2 = nn.Linear(512, 10)  # 输出10个类别

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        
        x = x.view(-1, 128 * 4 * 4)  # 展平为一维
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 将模型移到 GPU（如果可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)


In [6]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
def train_model(model, trainloader, criterion, optimizer, epochs=10):
    model.train()  # 设置模型为训练模式
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # 清空梯度
            outputs = model(inputs)  # 前向传播
            loss = criterion(outputs, labels)  # 计算损失
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数

            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader)}")

train_model(model, trainloader, criterion, optimizer, epochs=100)


Epoch 1/100, Loss: 1.364949301151973
Epoch 2/100, Loss: 0.9244127490788775
Epoch 3/100, Loss: 0.7313808593374994
Epoch 4/100, Loss: 0.5974984180439463
Epoch 5/100, Loss: 0.4728345434988856
Epoch 6/100, Loss: 0.36554144503896496
Epoch 7/100, Loss: 0.2721780803711975
Epoch 8/100, Loss: 0.19957456057486328
Epoch 9/100, Loss: 0.1471687484808895
Epoch 10/100, Loss: 0.11307135375592943
Epoch 11/100, Loss: 0.09982369187742929
Epoch 12/100, Loss: 0.08515703093910786
Epoch 13/100, Loss: 0.07770587292873798
Epoch 14/100, Loss: 0.07606355627954645
Epoch 15/100, Loss: 0.06813720111405272
Epoch 16/100, Loss: 0.06902980482708329
Epoch 17/100, Loss: 0.06743673075858714
Epoch 18/100, Loss: 0.059592960764418175
Epoch 19/100, Loss: 0.06259788001474836
Epoch 20/100, Loss: 0.05960740816846128
Epoch 21/100, Loss: 0.05756417805064872
Epoch 22/100, Loss: 0.04862442754601757
Epoch 23/100, Loss: 0.05551357114116799
Epoch 24/100, Loss: 0.05088543449751819
Epoch 25/100, Loss: 0.06139473351573273
Epoch 26/100, Lo

In [7]:
import gradio as gr
from PIL import Image

# 定义测试函数
def predict_image(image):
    model.eval()  # 设置模型为评估模式
    image = image.resize((32, 32))  # 调整图像大小为 32x32
    image = transforms.ToTensor()(image).unsqueeze(0).to(device)  # 转换并加一个维度
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    class_idx = predicted.item()

    # CIFAR-10 类别
    classes = [
        "airplane", "automobile", "bird", "cat", "deer", "dog", 
        "frog", "horse", "ship", "truck"
    ]
    return classes[class_idx]

# 创建 Gradio 界面
iface = gr.Interface(fn=predict_image, 
                     inputs=gr.Image(type="pil"),  # 仅指定类型为 PIL 图像
                     outputs="text")

# 启动 Gradio 前端界面
iface.launch()



* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://8ed230da4bab57aa9b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
